In [14]:
from bs4 import BeautifulSoup
import requests
import csv 
import os

In [15]:
def get_page_contents(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
    }

    page = requests.get(url, headers=headers)

    if page.status_code == 200:
        return page.text

    return None

In [16]:
def get_original_text(text):
    if(text=='Objective'):
        return 'Objectifs'
    elif(text=='Skills'):
        return 'Compétences'
    elif(text=='Jobs / Careers'):
        return 'Métiers'
    elif(text=='Sector of activity'):
        return 'Secteurs d’activité'
    elif(text=='Professional partnership'):
        return 'Partenariats professionnels'

In [ ]:
def get_semester(text):
    if(text=='Semester 1'):
        return 'Semestre 1' 
    elif(text=='Semester 2'):
        return 'Semestre 2'
    elif(text=='Semester 3'):
        return 'Semestre 3'
    elif(text=='Semester 4'):
        return 'Semestre 4'
    else:
        return text

In [24]:
def get_masters_MDSI(page_contents):
    soup = BeautifulSoup(page_contents, 'html.parser')
    structured_content = {}
    
    # Find the main div containing the sections
    parcours_find_div = soup.find('div', id='elementor-tab-content-7021', class_='elementor-tab-content elementor-clearfix')
    
    if parcours_find_div:
        # Extract headings and their corresponding content
        headings = parcours_find_div.find_all('h3')
        for heading in headings:
            key_text = heading.get_text(strip=True)
            values = []
            
            # Special case for "Matières par semestre"
            if key_text == 'Subjects per semester':
                semester_content = {}
                # Locate each semester section
                semesters = parcours_find_div.find_all('div', class_='elementor-accordion-item')
                matieres_par_semestre = {}
                for semester in semesters:
                    # Get the semester heading (e.g., "Semestre 1")
                    semester_heading = semester.find('a').get_text(strip=True)

                    # Find the content inside the semester section
                    content_div = semester.find('div', class_='elementor-tab-content elementor-clearfix')
                    if content_div:
                        # Extract list items
                        semester_content_list = [
                            li.get_text(strip=True) for li in content_div.find_all('li')
                        ]
                        if semester_heading and semester_content_list:
                            matieres_par_semestre[get_semester(semester_heading)] = semester_content_list

                structured_content['Matières par semestre'] = matieres_par_semestre
            elif(key_text == 'Content'):
                values = []
                for sibling in heading.find_next_siblings(['ul'], limit=1):
                    values.extend([li.get_text(strip=True) for li in sibling.find_all('li')])
                structured_content['Contenu'] = values
            else:
                # For other sections like "Objectifs," "Contenu," etc.
                for sibling in heading.find_next_siblings(['p', 'ul'], limit=3):  # Limit to avoid unrelated content
                    if sibling.name == 'p':
                        values.append(sibling.get_text(strip=True))
                    elif sibling.name == 'ul':
                        values.extend([li.get_text(strip=True) for li in sibling.find_all('li')])
                structured_content[get_original_text(key_text)] = values
    
    return structured_content

In [25]:
if __name__ == '__main__':
    url = 'https://www.esb.tn/programs-3/international-classes/professional-master-of-digital-management-information-systems/'
    page_contents = get_page_contents(url)
    result=[]   #keep result when appending all of the programs for now we will try to use it for csv test file

    if page_contents:
        result.append(get_masters_MDSI(page_contents))
        #print(get_masters_CCA(page_contents))
    else:
        print('Failed to get page contents.')

In [26]:
field_names = [
    'Objectifs', 'Contenu', 'Compétences', 'Métiers', 'Secteurs d’activité', 'Partenariats professionnels',
    'Semestre 1', 'Semestre 2', 'Semestre 3', 'Semestre 4'
]

# Flatten the data
flattened_result = {}
for dict in result:
    for key, value in dict.items():
        if key == 'Matières par semestre':
            # Extract each semester as a separate field
            for semester, courses in value.items():
                flattened_result[semester] = ', '.join(courses)
        elif isinstance(value, list):
            # Join list items with commas
            flattened_result[key] = ', '.join(value)
        else:
            flattened_result[key] = value
flattened_result.pop('Harmonization period')
#pop key none in flattened_result
flattened_result.pop(None)
print(flattened_result)
file_exists = os.path.exists('./masters.csv')
with open('./masters.csv', 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=field_names)
    
    if not file_exists or csvfile.tell() == 0:
        writer.writeheader()
    
    # Write the flattened result as a row
    writer.writerow(flattened_result)

{'Objectifs': 'TheProfessional Master in Digital Management and Information Systemstrains specialists in information systems management and the integration of information and communication technologies, capable of supporting companies in their digital transformation processes., TheProfessional Master in Digital Management & Information Systemsis available as a double degree in International Management with IMT-BS and in Management and Business Strategy with Ascencia Business School., MANAGEMENT:Business Process Management, Change Management, Project Management, Supply Chain, etc., DIGITAL:ERP, Business Intelligence, Big Data, SAP, CRM, SOFT SKILLS:Seminars, Personal Development, Business Games, Languages, etc., PROFESSIONAL PROJECTS:Internships, work placements, integrated projects, company visits, etc.', 'Contenu': 'MANAGEMENT:Business Process Management, Change Management, Project Management, Supply Chain, etc., DIGITAL:ERP, Business Intelligence, Big Data, SAP, CRM, SOFT SKILLS:Semi